# Q.1 Finding inverses.
Newton's method
```math
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}
```
our function is:
```math
f(x) = \frac{1}{x} - a = 0
```
```math
f'(x) = -\frac{1}{x^2}
```
then the Newton's method becomes
```math
x_{n+1} = x_n - \frac{\frac{1}{x_n} - a}{-\frac{1}{x_n^2}}
```
```math
x_{n+1} = x_n + x_n^2 \cdot (\frac{1}{x_n} - a)
```
```math
x_{n+1} = x_n + x_n  - a \cdot x_n
```
```math
x_{n+1} = x_n \cdot(2 - a x_n)
```

In [1]:
function inverses(x_n, a, NMAX)
    if NMAX <= 0 return x_n end
    inverses((x_n*(2-a*x_n)), a, NMAX-1)
end

inverses (generic function with 1 method)

In [2]:
x = 0.001
a = 0.1
NMAX = 1000
println([inverses(x, a, NMAX), 1/a])

[10.0, 10.0]


# Inverse of a matrix

In [3]:
function inverses_matrix(a, NMAX)
    I = zeros(size(a))
    for i in 1:size(a, 1) I[i, i] = 1 end
    x_n = 1/100 * a
    function inverses_matrix_internal(x_n, a, I, NMAX)
        if NMAX <= 0 return x_n end
        inverses_matrix_internal((x_n * (2 * I - a * x_n)), a, I, NMAX-1)
    end
    return inverses_matrix_internal(x_n, a, I, NMAX)
end

inverses_matrix (generic function with 1 method)

In [4]:
A = [11 0.486395; 5.43656 1.28172]
NMAX = 100

display(inverses_matrix(A, NMAX))
display(inv(A))
display(inverses_matrix(A, NMAX) * A)

2×2 Array{Float64,2}:
  0.111896  -0.0424628
 -0.474618   0.960313 

2×2 Array{Float64,2}:
  0.111896  -0.0424628
 -0.474618   0.960313 

2×2 Array{Float64,2}:
  1.0          0.0
 -8.88178e-16  1.0

I spent so much time at this looking for errors, and it tourn out to be that choise of X is that important, not only A!

# Q.2 Complex **i** as a 2x2 matrix.

In [5]:
imat = [ 0 -1; 1 0 ]
function print_imat(f)
    println(f[1,1], " + ", f[2, 1], "i")
end
print_imat(imat^2)
print_imat(exp(imat * pi))
print_imat(exp(imat * pi / 2))

-1 + 0i
-0.9999999999999999 + 2.351274992194946e-16i
1.501337291432472e-16 + 1.0i


In [12]:
(@isdefined σ₁) ? (isapprox(Main.σ₁, [0 1; 1 0]) ? println("σ₁ defined properly in Julia") : const σ₁ = [0 1; 1 0]) : const σ₂ = [0 -im; im 0]


2×2 Array{Complex{Int64},2}:
 0+0im  0-1im
 0+1im  0+0im

# Q.3 Pauli matrices

In [ ]:
using LinearAlgebra
using Random

# Define Pauli matrices - are predefined in Julia
@isdefined σ₁ ? (isapprox(Main.σ₁, [0 1; 1 0]) ? println("σ₁ defined properly in Julia") : const σ₁ = [0 1; 1 0]) : const σ₂ = [0 -im; im 0]

isapprox(Main.σ₂, [0 -im; im 0]) ? println("σ₂ defined properly in Julia") : const σ₂ = [0 -im; im 0]
isapprox(Main.σ₃, [1 0; 0 -1]) ? println("σ₃ defined properly in Julia") : const σ₃ = [1 0; 0 -1]
#isapprox(Main.σ, [σ₁, σ₂, σ₃]) ? println("σ defined properly in Julia") : const σ = [Main.σ₁, Main.σ₂, Main.σ₃]
const σ = [σ₁, σ₂, σ₃]  # Array of Pauli matrices

# Part (a): Verify the relations
function verify_relation_1(a::Vector{Float64}, b::Vector{Float64})
    # Compute (σ · a) and (σ · b)
    σ_dot_a = sum(ai * σi for (ai, σi) in zip(a, σ))
    σ_dot_b = sum(bi * σi for (bi, σi) in zip(b, σ))
    
    # Compute the left and right-hand sides of the relation
    lhs = σ_dot_a * σ_dot_b
    rhs = (dot(a, b) * I(2)) + im * sum((cross(a, b)[i] * σ[i]) for i in 1:3)
    
    return isapprox(lhs, rhs) # check equality
end

function compute_D(ϕ::Float64, n::Vector{Float64})
    # Compute the SU(2) matrix
    σ_dot_n = sum(ni * σi for (ni, σi) in zip(n, σ))
    D = cos(ϕ / 2) * I(2) - im * sin(ϕ / 2) * σ_dot_n
    return D
end

# Part (b): Restrictions and degrees of freedom
function check_su2_matrix(D::Matrix{ComplexF64})
    # Check determinant and unitarity
    det_condition = isapprox(det(D), 1) #  matrix unimodular
    hermitian_condition = isapprox(D' * D, I(2)) # matrix unitarity
    # if both -> spetial unitarity 
    return det_condition && hermitian_condition
end

# Part (c): Generate random SU(2) matrices
function generate_random_su2()
    # Randomly sample angles
    ϕ = rand() * 2π  # Angle in [0, 2π]
    θ = rand() * π   # Angle in [0, π]
    ζ = rand() * 2π  # Angle in [0, 2π]
    
    # Generate a random unit vector
    n = [sin(θ) * cos(ζ), sin(θ) * sin(ζ), cos(θ)]
    
    # Compute and return the SU(2) matrix
    return compute_D(ϕ, n)
end

# Part (d): Levi-Civita symbol
function levi_civita(i::Int, j::Int, k::Int)
    # Compute the commutator [σ_i, σ_j]
    commutator = σ[i] * σ[j] - σ[j] * σ[i]
    
    # Compute the trace with σ_k
    return real(tr(commutator * σ[k])) / (8im)
end

# Alternative method for Levi-Civita symbol
function levi_civita_alternative(i::Int, j::Int, k::Int)
    if i == j || j == k || k == i
        return 0
    end
    if (i, j, k) in [(1, 2, 3), (2, 3, 1), (3, 1, 2)]
        return 1
    end
    if (i, j, k) in [(3, 2, 1), (1, 3, 2), (2, 1, 3)]
        return -1
    end
    return 0
end

# Example usage
a = [1.0, 2.0, 3.0]
b = [4.0, 5.0, 6.0]
ϕ = π / 4
n = [0.0, 0.0, 1.0]

# Verify relations
println("Verify Relation 1:", verify_relation_1(a, b))
D = compute_D(ϕ, n)
println("SU(2) Matrix D:\n", D)
println("Check SU(2) Properties:", check_su2_matrix(D))

# Generate random SU(2)
random_su2 = generate_random_su2()
println("Random SU(2) Matrix:\n", random_su2)

# Compute Levi-Civita symbol
println("Levi-Civita (method 1):", levi_civita(1, 2, 3))
println("Levi-Civita (alternative):", levi_civita_alternative(1, 2, 3))


UndefVarError: [91mUndefVarError: σ₁ not defined[39m

# Q.4 Legendre Transform

In [18]:
H = px^. - L

Base.Meta.ParseError: ParseError:
# Error @ c:\Users\Tomek\source\repos\NumQM\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X46sZmlsZQ==.jl:1:8
H = px^. - L
#      ╙ ── invalid identifier

# TODO
# Q.5  Gaussian integral

In [34]:
# simpson_req is adaptive recursive Simpson's rule | interesting case: sin(x) in [0, 2pi]
function simpsons_req(f, A, a, b, TOL=2^(-8))  
    function integrate_(f, A, a, fa, b, fb, TOL)
        n = (a+b)/2
        fn = f(A, n)
        res = (b-a)/6 * (fa + 4*fn + fb)
        if res < TOL return res 
        else return integrate_(f, A, a, fa, n, fn, TOL) + integrate_(f, A, n, fn, b, fb, TOL) end
    end
    return integrate_(f, A, a, f(A, a), b, f(A, b), TOL)
end

# iterative solution for usage to specyfic case functions
function simpsons_iter(f, A, a, b, N)
    step = (b-a)/N
    res = 0
    _a = a
    _b = a + step
    _f_a = f(A, _a)
    _f_b = f(A, _b)
    
    for _ in 1:N
        res += (_b-_a)/6*(_f_a + 4*f(A, (_a+_b)/2) + _f_b)
        _a = _b
        _b = _a + step
        _f_a = _f_b
        _f_b = f(A, _b)
    end
    return res
end

function gaussian(A, x)
    return exp(-A*(x^2))
end

gaussian (generic function with 1 method)

In [36]:
a = -2^16
b = 2^16
A = 1
TOL = 2^(-16)

#result = simpsons_req(gaussian, A, a, b, TOL)
result = simpsons_iter(gaussian, A, a, b, 2^18)

println(result, "\t", sqrt(pi/A), "\t", result - sqrt(pi/A))  

1.7724538509055159	1.7724538509055159	0.0


In [47]:
# 1D Gaussian by dr Pom Man Lo :https://numqm.netlify.app/lessons/gaussianintegrals

N = 100
rv = range(-1.0, stop=1.0, length=N)
dr = 2.0/(N-1)

# map (-1, 1) to (-Inf, Inf)
xvf = r -> r/(1-r^2)
xwf = r -> (1+r^2)/(1-r^2)^2

f = x -> exp(-x^2)

ret = 0.0

for rr in rv[1:N-1]
    #println(rr+dr/2,"\t", xwf(rr+dr/2), "\t", xvf(rr+dr/2))
    ret += dr * xwf(rr+dr/2) * f(xvf(rr+dr/2))
end

println([ret, sqrt(pi)])

[1.77245, 1.77245]


Usage of Gaussian integral (quantum mechanics)

In [39]:
A = 1 # global parameter
f = x -> exp(-A/2*(x^2))
#simpsons_req(f, A, a, b, TOL) 
f_help_res(A, x) = exp(-A*x^2)
result_help = simpsons_req(f_help_res, A, -2^8, 2^8)
fu_x(A, x) = f(x)*x*f(x)
#funkcyjka = A, x -> f(x)*x*f(x)
d_x_b = simpsons_req(fu_x , A, -2^8, 2^8) / result_help
println("<x> =", d_x_b)

fu_xx(A, x) = f(x)*x^2*f(x)
#d_xx_b = simpsons_req(fu_xx, A, -2^8, 2^8) / result_help
d_xx_b = simpsons_iter(fu_xx, A, -2^8, 2^8, 2^18) / result_help
println("<x^2> =", d_xx_b)


# one of the derivative methods described in lecture
function derivative_1ord(f, A, x0, dx=2^(-8))
    return 1/12/dx * 
        (-f(A,x0+dx+dx)
         -8*f(A,x0-dx)
         +8*f(A,x0+dx)
         +f(A,x0-dx-dx))
end

fu_help(A, x) = f(x)
p = x -> -1im*derivative_1ord(fu_help, A, x)
fu_p(A, x) = real(f(x)*p(x)) # not sure about this
d_p_b = simpsons_req(fu_p, A, -2^8, 2^8,) / result_help
println("<p> =", d_p_b)

function derivative_2ord(f, x0, dx=2^(-8))
    return (f(x0 + dx) - 2*f(x0) + f(x0 - dx))/dx/dx
end

pp = x -> -derivative_2ord((x -> f(x)), x)
fu_pp(A, x) = real(f(x)*pp(x))
d_pp_b = simpsons_req(fu_pp, A, -2^8, 2^8) / result_help
print("<p^2> =", d_pp_b)

<x> =0.0
<x^2> =0.49998885252366476
<p> =0.0
<p^2> =0.4972957813044254

In [40]:
(d_pp_b - d_p_b^2) * (d_xx_b - d_x_b^2)

0.248642347059259